# Spike Finder

Iterate through folders, looking for possible spikes in all supported GPS file types.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

This is very crude and simply looks for Doppler speeds in excess of 30 m/s (i.e. 108 km/h or 58.32 knots).

Such speeds are currently unrealistic for a windsurfer.

In [1]:
import os
import sys

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [2]:
def findSpikes():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions')

    errors = {}
    spikes = {}

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext and ext in ('.fit', '.gpx', '.nmea', '.oao', '.sbn', '.sbp', '.ubx'):
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Ignore GT-11 files
                    if not ('firmware' in reader.header and reader.header['firmware'].startswith('V1.62')):
                        if 'sog' in reader.data:
                            maxSpeed = reader.data['sog'].max()
                            if maxSpeed > 30:
                                msg = '{} m/s ({:.02f} knots / {:.02f} km/h)'.format(
                                    maxSpeed, maxSpeed * 3600 / 1852, maxSpeed * 3600 / 1000)
                                spikes[filePath.replace(projdir + '/', '')] = msg
                                print('S', end='')
                            else:
                                print('.', end='')
                    else:
                        print('.', end='')
                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

    if len(spikes) > 0:
        print(os.linesep * 2 + 'Spikes:')
        for spike in spikes:
            print()
            print(spike)
            print(spikes[spike])

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    findSpikes()
    
    print(os.linesep + 'All done!')
    
    str(12)

............................................................................................SS.................................................................................................................................................S........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................